In [ ]:
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo
from scipy import signal
import scipy.cluster.hierarchy as spc
from pandas import read_excel
from ipywidgets import widgets
from ipywidgets import interactive, HBox, VBox
import plotly.io as pio
init_notebook_mode(connected=True)

keywords:

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360)

#list topics 
kw_list = []

kw_list.append(["/m/032nch"]) #cruauté envers les animaux, Sujet
kw_list.append(["l214"])
kw_list.append(["/m/047v0jr"]) #abattage
kw_list.append(["/m/07628"]) #spécisme
kw_list.append(["/m/07_lq"]) #cuisine vegetarienne
kw_list.append(["/m/07_jd"]) #végétarisme
kw_list.append(["/m/07_hy"]) #véganisme
kw_list.append(["/m/020953"]) #substitut de viande

#list country https://abbreviations.yourdictionary.com/articles/abbreviations-european-union.html
country_list = ["FR","BE","BG","CY","CZ","DE","DK","EE","ES","FI","AT","GB","GR","HR","HU","IE","IT","LT","LU","LV","MT","NL","PL","PT","RO","SE","SI","SK"]
country_list_iso = ["FRA","BEL","BGR","CYP","CZE","DEU","DNK","EST","ESP","FIN","AUT","GBR","GRC","HRV","HUN","IRL","ITA","LTU","LUX","LVA","MLT","NLD","POL","PRT","ROU","SWE","SVN","SVK"]


for x in range(len(kw_list)):
    
    #adding just the first one
    pytrends.build_payload(kw_list[x], cat=0, timeframe='all', geo=country_list[0], gprop='')
    pytrends.interest_over_time()
    newDataframe = pytrends.interest_over_time()[kw_list[x]]
    listColumn = []
    listColumn.append(country_list[0])
        
    for y in range(1, len(country_list)): 
        pytrends.build_payload(kw_list[x], cat=0, timeframe='all', geo=country_list[y], gprop='')
        ledata = pytrends.interest_over_time()
        if not ledata.empty:
            ledata.drop(ledata.columns[len(ledata.columns)-1], axis=1, inplace=True)
            newDataframe = pd.concat([newDataframe, ledata], axis=1, join='inner')
            listColumn.append(country_list[y])

    newDataframe.columns = listColumn
    newDataframe = newDataframe[19:] #remove first strange Data
    newDataframe.to_pickle("./data_vinc/GoogleTrend"+str(x)+".pkl") 



In [ ]:
df_animal_cruelty = pd.read_pickle("./data_vinc/GoogleTrend0.pkl")
df_l214 = pd.read_pickle("./data_vinc/GoogleTrend1.pkl")
df_abattage = pd.read_pickle("./data_vinc/GoogleTrend2.pkl")
df_specism = pd.read_pickle("./data_vinc/GoogleTrend3.pkl")
df_vegan_cook = pd.read_pickle("./data_vinc/GoogleTrend4.pkl")
df_vegetarism = pd.read_pickle("./data_vinc/GoogleTrend5.pkl")
df_veganism = pd.read_pickle("./data_vinc/GoogleTrend6.pkl")
df_meat_substitute = pd.read_pickle("./data_vinc/GoogleTrend7.pkl")

In [ ]:
def plot_lines(df,value_smooth):
    fig = go.Figure()
    for x in df.columns:
        if value_smooth == 0:
            fig.add_trace(go.Scatter(x=df.index, y=df[x], mode='lines',name=x))
        else:
            fig.add_trace(go.Scatter(x=df.index, y=signal.savgol_filter(df[x],value_smooth, 3), mode='lines',name=x))
    fig.show()

In [ ]:
plot_lines(df_animal_cruelty,25)